### report of implementations

In [125]:
from __future__ import absolute_import, division, print_function, unicode_literals
from classes2048 import state, tbl
from time import sleep
import tensorflow as tf
import numpy as np
from functions import mv
from pynput.keyboard import Key, Controller
import math
from IPython.display import clear_output
import os
keys= [Key.right,Key.down,Key.up,Key.left]

In [132]:
from classes2048 import state, tbl

In [133]:
game=state()
table=tbl(game)
keyboard = Controller()

In [139]:
game.props=props
table.props=props

In [140]:
game.statepredict()

'going'

In [141]:
table.gettbl()

,0,1,2,3
0,NaN,2.0,8.0,2.0
1,4.0,32.0,16.0,128.0
2,8.0,64.0,8.0,2.0
3,4.0,16.0,4.0,16.0


In [ ]:
game.getprops()

### Hyperparameters 

In [103]:
# num_input_neurons = len(env.reset())
# num_ouptut_neurons = env.action_space.n

num_input_neurons = 16
num_ouptut_neurons = 4
num_hidden_layer_neurons_1 = 256

gamma = 0.99
max_epsilon = 0.9
epsilon = max_epsilon
min_epsilon = 0.01
epsilon_decay_factor = 0.001

num_episodes = 10

learning_rate = 0.001

In [142]:
table.onehot()

AttributeError: 'tbl' object has no attribute 'onehot'

### training the Q-network

In [145]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, kernel_initializer='random_uniform', activation='relu', input_dim = 16),
    tf.keras.layers.Dense(128, kernel_initializer='random_uniform',activation='relu'),

    tf.keras.layers.Dense(4, kernel_initializer='random_uniform',activation='relu')
#       tf.keras.layers.Dense(128, activation='relu', input_dim = 16),
#       tf.keras.layers.Dense(4, activation='relu')
    
])
opt=tf.keras.optimizers.Adam(lr=learning_rate)

model.compile(optimizer=opt,
              loss='mse',       # mean squared error
              metrics=['mae'])


In [9]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

'going'

In [143]:
    def onehot(t, log=True, sub=1):
        if log:
            return np.reshape(np.array( list( map(np.log2,t.tbl.replace(np.nan, sub).values.flatten()) ) ), [1, 16])
        else:
            return np.reshape(t.tbl.replace(np.nan, sub).values.flatten(),[1, 16])

In [208]:
#reward_list = []
num_episodes=1
sleep(2)
for i in range(num_episodes):
    table.gettbl()
    tablestate = onehot(table)
    while game.statepredict()=='going':
        action, q_vals = np.array([np.argmax(model.predict(tablestate))]), model.predict(tablestate)
        
        if np.random.rand(1) < epsilon:
            action[0] = np.random.randint(4)
            
        mv(keys[action[0]] )
        sleep(0.3)
        table.gettbl()
        next_tablestate=onehot(table)
        
        if np.array_equal(next_tablestate,tablestate):
            reward=-100
        else:
            reward = tablestate.sum()-next_tablestate.sum()
        
        if game.statepredict()=='lost':
            reward=-1000

        max_q=np.max(model.predict(tablestate))
        target_q_vals = q_vals
        target_q_vals[0, action[0]] = reward + gamma * max_q
        
        print(table.gettbl(),reward, keys[action[0]])
        model.fit(tablestate, target_q_vals)
        

        clear_output(wait=True)

        tablestate = next_tablestate
        reward_so_far += reward
        sleep(0.1)

    # epsilon decay
    game.statepredict(save=True)
    keyboard.press(Key.f5)
    sleep(0.02)
    keyboard.release(Key.f5)
    sleep(3)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * math.exp(-epsilon_decay_factor * i)

    reward_list.append(reward_so_far)


KeyboardInterrupt: 

In [167]:
checkpoint_path = "checkpoints/"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create checkpoint callback
# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

model.save_weights('./checkpoints/')

In [204]:
for i in range(100):
    sleep(2)
    table.gettbl()
#         tablestate = table.onehot()
    tablestate = onehot(table)
    print(model.predict(tablestate))


[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]


KeyboardInterrupt: 

In [200]:
tablestate

array([[0., 0., 0., 0., 0., 1., 4., 1., 0., 4., 3., 4., 1., 1., 2., 5.]])

array([[0., 0., 0., 0.]], dtype=float32)

In [166]:
reward_list

[-2436.0,
 -2242.0,
 -3343.0,
 -4749.0,
 -1539.0,
 -2648.0,
 -2145.0,
 -741.0,
 -742.0,
 -1240.0,
 -2344.0,
 -643.0,
 -1138.0,
 -2543.0,
 -1139.0,
 -2846.0,
 -2342.0,
 -3845.0,
 -837.0,
 -1839.0,
 -2345.0,
 -1841.0,
 -2446.0,
 -1544.0,
 -1542.0,
 -1643.0,
 -2649.0,
 -1844.0,
 -4749.0,
 -2945.0]

In [154]:
game.statepredict(save=True)

'lost'

In [153]:
table.gettbl()

,0,1,2,3
0,2.0,4.0,32.0,NaN
1,128.0,16.0,64.0,2.0
2,16.0,32.0,8.0,NaN
3,4.0,32.0,8.0,NaN


In [156]:
game.fit()
game.save()

/home/ainur/venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


## testing the network

In [159]:
num_episodes = 2
total_reward = []

# model = create_model()
# model.load_weights(latest)
# loss, acc = model.evaluate(test_images, test_labels)
 
sleep(2)
table.gettbl()

for _ in range(num_episodes):
    while game.statepredict()=='going':
        tablestate = np.reshape(table.tbl.replace(np.nan, 1).values.flatten(), [1, num_input_neurons])
#         action = sess.run(best_action, feed_dict={input_obs: state})
        action= np.array([np.argmax(model.predict(tablestate))])
        mv(keys[action[0]])
        sleep(2)
        table.gettbl()
        next_tablestate = np.reshape(table.tbl.replace(np.nan, 1).values.flatten(), [1, num_input_neurons])
        if np.array_equal(next_tablestate,tablestate):
            reward=-100
        else:
            reward = tablestate.prod()/next_tablestate.prod()
            
       # state, reward, done, _ = env.step(action[0])
        total_reward.append(reward)

        sleep(0.1)


print("Average reward per episode: {}".format(total_reward/float(num_episodes)))

KeyboardInterrupt: 

In [160]:

reward

-100

In [152]:
model.predict(state)[0]

array([0.5267891, 7.842474 , 0.       , 8.211992 ], dtype=float32)

In [48]:
q=np.array([[1]*16])

In [51]:
state[0].replace(np.nan,0)

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [94]:
state0=state

In [96]:
np.array_equal(state0,state)

True

In [ ]:
num_episodes = 1
total_reward = 0

for _ in range(num_episodes):
    done = False
    while game.statepredict()=='going':
        
        state = np.reshape(table.tbl.values.flatten(), [1, num_input_neurons])
        action = sess.run(best_action, feed_dict={input_obs: state})
        state, reward, done, _ = env.step(action[0])
        
        
#         state = np.reshape(state, [1, num_input_neurons])
#         action = sess.run(best_action, feed_dict={input_obs: state})
#         state, reward, done, _ = env.step(action[0])
        total_reward += 1

        env.render()
        time.sleep(0.1)

    env.reset()